In [29]:
import os
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from tqdm import tqdm
from math import log
import pickle
import matplotlib.pyplot as plt

In [59]:
X = []
y = []
z = []
check = ['us','indian','england','canada','australia','scotland','african']
phones_all = ['ah', 't', 'n', 'ih', 'd', 's', 'r', 'l', 'iy', 'dh', 'k', 'm', 'z', 'eh', 'ae', 'w', 'er', 'ay', 'hh', 'uw', 'p', 'b', 'v', 'f', 'ey', 'ow', 'aa', 'ao', 'ng', 'y', 'g', 'sh', 'th', 'aw', 'uh', 'jh', 'ch', 'oy', 'zh']
phones = ['iy','ih','ix','ey','eh','er','ae','aa','ao','ay','aw','ah','ax','axr','ow','oy','uh','uw','ux']
phones = [x for x in phones if x in phones_all]
print(phones)
print(len(phones))

['iy', 'ih', 'ey', 'eh', 'er', 'ae', 'aa', 'ao', 'ay', 'aw', 'ah', 'ow', 'oy', 'uh', 'uw']
15


##### Set priors, used uniform prior

In [35]:
prior = [0.014707921313730166, 0.02592187941944751, 0.0915695659166662, 0.014447260566465718, 0.00692137473587302, 0.021474008795913727, 0.01883967145653898, 0.00446450854357194, 0.04471718308921807, 0.031783973246224576, 0.027630039210031557, 0.025306276378035727, 0.016743293531731287, 0.016998408305649682, 0.010243412770158227, 0.019993233912517817, 0.0645828596147767, 0.038084198967339764, 0.004708531370798232, 0.030159003055831312, 0.03876080771555812, 0.029188457720272197, 0.06547021535014502, 0.012056946054317263, 0.006250311961000715, 0.014713467287076218, 0.0017913493907748279, 0.019272257377531042, 0.03941523257039227, 0.044068304207729975, 0.007376144550249292, 4.436778676841679e-05, 0.07593546705414533, 0.007182035483137468, 0.00657752438841779, 0.017131511665954932, 0.025467109605071238, 0.011929388667358065, 0.028367653665056485]
prior = np.array(prior)/sum(prior)
print(prior)

[1.50035642e-02 2.64429333e-02 9.34101993e-02 1.47376639e-02
 7.06050080e-03 2.19056563e-02 1.92183664e-02 4.55424931e-03
 4.56160400e-02 3.24228606e-02 2.81854287e-02 2.58149560e-02
 1.70798493e-02 1.73400921e-02 1.04493149e-02 2.03951165e-02
 6.58810351e-02 3.88497267e-02 4.80317723e-03 3.07652270e-02
 3.95399360e-02 2.97751728e-02 6.67862275e-02 1.22993019e-02
 6.37594904e-03 1.50092216e-02 1.82735718e-03 1.96596477e-02
 4.02075154e-02 4.49541181e-02 7.52441191e-03 4.52596205e-05
 7.74618405e-02 7.32640107e-03 6.70973874e-03 1.74758710e-02
 2.59790222e-02 1.21691805e-02 2.89378699e-02]


##### data loading

In [15]:
def load_data(phones,check,layer):
    X = {i:[] for i in check}
    z = {i:[] for i in check}
    y = {i:[] for i in check}
    path = 'phones/avg/test/'
    #layer = 'rnn_2'
    layer_path = os.path.join(path,layer)
    print(layer_path)
    start = True
    for r,d,fs in os.walk(layer_path):
        if(start):
            start = False
            continue
        accent = r.split('/')[-1]
        print(accent)
        if(accent not in check): continue
        phn_count = {x:0 for x in phones}
        for f in fs:
            phn = f.split('_')[-2]
            if(phn in phones):
                rep = np.load(os.path.join(r,f))
                phn_count[phn] += 1
                X[accent].append(rep)
                y[accent].append(check.index(accent))
                z[accent].append(phones.index(phn))
                #y.append(accent)
        #print(phn_count)
    return X,y,z
    #print(f[:10])
        

In [5]:
X_dict, y_dict, z_dict = load_data(phones,check,'rnn_4')

phones/avg/test/rnn_4
canada
england
australia
us
indian
african
scotland


In [6]:
X = []
y = []
for a in check:
    X.extend(X_dict[a])
    y.extend(y_dict[a])
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)



(191216, 1024)
(191216,)


##### Clustering

In [7]:
kmeans = MiniBatchKMeans(n_clusters=5000).fit(X)

/home/archiki/.local/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)
/home/archiki/.local/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)
/home/archiki/.local/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)


### Probability Calculations

In [8]:
prior_prob = np.ones(len(check))/len(check)
print(prior_prob)
posterior = {i:np.zeros(5000) for i in check}
#print(posterior)

[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]


In [9]:
# compute posterior
for a in check:
    clusters = kmeans.predict(X_dict[a])
    unique, counts = np.unique(clusters, return_counts=True)
#     print(unique)
#     print(counts)
    counts = counts/sum(counts)
    #print(unique.shape)
    for j in range(unique.shape[0]):
        #print(j)
#         print(np.where(unique == j)[0])
        posterior[a][unique[j]] = counts[j]
#     print(posterior[a])
#     break
        
    


In [10]:
print(prior_prob[2])
#print(posterior[2])
joint = {i:posterior[i]*prior_prob[check.index(i)] for i in check}
#print(joint['indian'])

0.14285714285714285


In [11]:
marginal_y = np.zeros(5000)
for i in range(5000):
    for a in check:
        marginal_y[i] += joint[a][i]

In [25]:
#print(marginal_y)
print(log(len(phones),2))

5.285402218862249


In [13]:

##### calculate mutual information
MI = 0
eps = 10e-12
for i in range(5000):
    for a in check:
        #print(joint[a][i])
#         fr = joint[a][i]/(prior_prob[check.index[a]]*marginal(y))
#         print(fr)
#         print(joint[a][i])
#         print(log(eps + joint[a][i]/(prior_prob[check.index(a)]*marginal_y[i] + eps), 2))
        MI += joint[a][i]*log((joint[a][i] + eps)/(prior_prob[check.index(a)]*marginal_y[i] + eps), 2)
print(MI)

0.17190594125098443


In [110]:
#print(type(kmeans))
pickle.dump(kmeans,open('kmeans_spec.pkl','wb'))

### MI between representations and phones


In [14]:
spec 0.3107413322810069
conv 0.1902783188322894
rnn_0 0.1927891895978321
rnn_1 0.13806864211728404
rnn_2 0.1723223791030824
rnn_3 0.1613138923844542
rnn_4 0.17190594125098443





In [ ]:
layers = ['spec','conv','rnn_0', 'rnn_1', 'rnn_2', 'rnn_3', 'rnn_4']
num_clust = 1000
MI_layers = {a:[] for a in check}
for layer in layers:
    X_dict, y_dict, z_dict = load_data(phones,check,layer)
    X = []
    y = []
    z = []
#     for a in check:
#         X.extend(X_dict[a])
#         y.extend(y_dict[a])
    for a in check:
        
        X = np.array(X_dict[a])
        y = np.array(y_dict[a])
        z = np.array(z_dict[a])
        print(X.shape)
        print(z.shape)
        
        kmeans =  MiniBatchKMeans(n_clusters=num_clust).fit(X)
        #prior_prob = prior
        prior_prob = np.ones(len(phones))/len(phones)
        #print(prior_prob)
        posterior = {i:np.zeros(num_clust) for i in phones}
        # compute posterior
        for p in phones:
            
            #print(X[z == phones.index(p)].shape)
            
            clusters = kmeans.predict(X[z == phones.index(p)])
            unique, counts = np.unique(clusters, return_counts=True)
        #     print(unique)
        #     print(counts)
            counts = counts/sum(counts)
            #print(unique.shape)
            for j in range(unique.shape[0]):
                #print(j)
        #         print(np.where(unique == j)[0])
                posterior[p][unique[j]] = counts[j]
        #     print(posterior[a])
        #     break
        joint = {i:posterior[i]*prior_prob[phones.index(i)] for i in phones}
        marginal_y = np.zeros(num_clust)
        for i in range(num_clust):
            for p in phones:
                marginal_y[i] += joint[p][i]
        MI = 0
        eps = 10e-12
        for i in range(num_clust):
            for p in phones:
                #print(joint[a][i])
        #         fr = joint[a][i]/(prior_prob[check.index[a]]*marginal(y))
        #         print(fr)
        #         print(joint[a][i])
        #         print(log(eps + joint[a][i]/(prior_prob[check.index(a)]*marginal_y[i] + eps), 2))
                MI += joint[p][i]*log((joint[p][i] + eps)/(prior_prob[phones.index(p)]*marginal_y[i] + eps), 2)
        print(layer,a, MI)
        MI_layers[a].append(MI)


    #print(posterior)




In [71]:
for a in check:
    print(a, MI_layers[a])
print(len(phones))

us [1.7915903050232593, 1.4611121843872, 1.5314252641986181, 2.0713711728558573, 2.6226536614728975, 2.576459346789232, 2.0383168861433876]
indian [1.7420024989632603, 1.325897680367223, 1.317594701715221, 1.7946481138399377, 1.6021141950958733, 1.3240542680243175, 1.9400388669970483]
england [1.7301636352786793, 1.2611593545382, 1.103424900633968, 1.8465019143780255, 1.6480655069217636, 2.2098659325865193, 2.2719092189660652]
canada [1.6484383103022004, 1.2820428302749434, 1.602191309965651, 2.158439627135968, 2.577331392817603, 2.713522450152562, 2.7035493792961507]
australia [1.6176451697675633, 1.3209664022203487, 1.3887311173637673, 1.8788460741282638, 2.3584084487239756, 2.328134853360021, 1.839986371281332]
scotland [1.6511738220493428, 1.3918310281321944, 1.4773989904103464, 1.8626307969178684, 2.1436552798772412, 2.1413478615059804, 2.0868136652732887]
african [1.7298747656789462, 1.3585588032384015, 1.4724245192444407, 2.0531028014403474, 2.4228996167607195, 2.433704335368076

In [ ]:
# vowels
# 500
# us [1.4387014648916878, 1.22312647739189, 1.2091937397106, 1.9491720532510297, 2.3660564046316415, 2.2878018210485735, 2.3454944133182956]
# indian [1.383546287446897, 0.9911436960404194, 1.1891593473038122, 1.5695978314340466, 1.6926381055173183, 1.6631155366157542, 1.68250545764955]
# england [1.2500726368155575, 1.1114423567554015, 1.1858304245186646, 1.5590731879100406, 1.9682328061064267, 1.84191287837616, 2.0573561991039053]
# canada [1.3083928908728155, 1.183495531312932, 1.1674821139373552, 1.9640804984458977, 1.9909322912314686, 2.318892609959518, 2.342447716561997]
# australia [1.25787626905837, 1.146255685994345, 1.2236181134925543, 1.7994574673911194, 2.049952217838728, 2.0135172302054953, 2.0571247395804315]
# scotland [1.4362301494538694, 1.1788679967694553, 1.2678928288936904, 1.389572966180646, 1.8516972500135285, 1.822431527523989, 1.7972391478528047]
# african [1.2262964001630001, 1.1701817430623087, 1.2825081530111302, 1.837690180009105, 1.6794005744772778, 1.4586716329287264, 1.4825447338685518]




In [43]:
# uniform
# us [1.8765280962417679, 1.4597635695178734, 1.9367714065609969, 2.792184650787109, 3.497082234976212, 3.2117770292718784, 2.948399689091605]
# indian [2.009438557974934, 1.3240284412852115, 1.5516268074426625, 2.286566869789922, 2.747366792803952, 2.294044869738722, 2.151009146979473]
# england [1.8314885654944277, 1.6379154936413647, 1.6947555426942422, 2.513079063933781, 3.1734743448729215, 2.7959262140242194, 2.8220981998300116]
# canada [1.9545243649646158, 1.608810609718501, 1.5487204291765215, 2.8905580108337037, 3.648665311751061, 3.1980517184003476, 3.0089546019556788]
# australia [1.8627109954673233, 1.5775635076392192, 1.801189632210137, 1.6230165658980067, 3.1857718034967033, 2.8502065211338117, 2.7137173125398903]
# scotland [2.171596827613306, 1.3554008366825725, 1.8706006406342819, 2.583980163413102, 3.0780231324419587, 1.4551091157450056, 2.5446346712666403]
# african [1.8647140878804473, 1.7071637923609817, 1.8283843066062218, 2.7035577226476586, 3.2960435929159213, 2.8560224127639606, 2.6318821461794313]

# uniform 500
# us [1.8434262101618228, 1.5745103761981654, 1.7984697832077485, 2.679742804733628, 3.228983907898553, 2.7875026153847573, 2.6472836603629553]
# indian [1.6467067476601673, 1.3019206470823628, 1.516094075967711, 2.105331507013192, 2.472458246995422, 2.0668339779820104, 0.9114223120433371]
# england [1.6443838600497922, 1.2270248687339542, 1.2728659574613144, 2.4971455012768295, 3.010701810584033, 2.554937870501281, 2.420212253507664]
# canada [1.670203294581223, 1.6244158301265976, 1.8345812441801495, 2.723283503107229, 1.6717639234851767, 3.0377667296477764, 2.691632968436616]
# australia [1.7743251006464515, 1.51266430199685, 1.6633238673805217, 2.373240506280682, 1.1272411621129208, 2.5313473234856567, 2.2816592554214434]
# scotland [1.5882350195245123, 1.5221501577710699, 1.5596688522145878, 2.4037287197222255, 2.7857829354101047, 2.341912435028621, 2.1896744361536444]
# african [1.5475371638512383, 1.4383662078803465, 1.6754115113751311, 2.5663293669322638, 3.1809397570390727, 0.9984550001815964, 2.5577537212735013]

# uniform 1500
# us [2.098315109749764, 1.5295358578289504, 2.036841059915939, 2.9964164931600688, 3.638400193077257, 3.271941131720866, 3.2895216361599284]
# indian [1.9649739436783815, 1.6220370294847657, 1.367703255641711, 2.3442876923930562, 2.8948830111911876, 1.43160170694874, 2.4821594392782367]
# england [2.3174902411238043, 1.6498083837069384, 1.805874722564403, 2.5936935939969383, 3.237204431894776, 2.989185620313446, 2.9818950879682933]
# canada [2.1509240378034358, 1.547922820457808, 2.074022791470695, 3.056125229804196, 3.7193352752149806, 3.4037273809314903, 2.26292134518766]
# australia [2.053960099329522, 1.5501989090356756, 1.8379728308290908, 2.6270251533894857, 3.3959228284139384, 2.938074835787055, 2.9419188708046633]
# scotland [2.1060989007674196, 1.7634567984185456, 1.9926340040196289, 2.801877037849214, 1.7751892423201496, 2.877989619980491, 2.620844635220367]
# african [2.282543059410469, 1.6109217545414358, 1.967706426375999, 2.8675972170467454, 3.434614157658132, 3.0850348808385393, 2.0299149784906296]

# uniform 2000
# us [2.4821262314733628, 1.8744720835641042, 2.046807324933854, 2.9007632402357904, 3.790268620387887, 3.3980740900158097, 3.4487635276349398]
# indian [2.1339303611951115, 1.6707330469600479, 1.8910532547455956, 2.495490991858255, 1.5598944248176223, 2.685352408689403, 2.6286428139697957]
# england [2.1983951053590354, 1.5425956291746834, 1.373951876695004, 2.749638484068691, 3.3517385978301717, 3.2001900401061145, 3.038177905177629]
# canada [2.3150468984556567, 1.9681398364557199, 2.1855386538770607, 3.1225216030431193, 3.8444136425614364, 3.5909169722153558, 3.494232996617487]
# australia [2.252032674680827, 1.8160426085490107, 2.056058922743928, 2.8372575723551248, 3.4530051404007147, 2.064954872178409, 3.143187229988373]
# scotland [2.2875494803242846, 1.8700778083027485, 2.0785178851726087, 2.8135502156750714, 3.309765501499925, 2.9689887827966683, 2.8785056077286573]
# african [2.437988013233287, 1.86280468336028, 1.5228218503507596, 2.9902225515519776, 3.4949845225796086, 3.2977029695314526, 3.2298852518939847]




In [ ]:
objects = ['spec', 'conv', 'rnn_0', 'rnn_1', 'rnn_2', 'rnn_3', 'rnn_4']
#objects.sort()
        #print()
y_pos = np.arange(len(objects))
fig = plt.figure(figsize = (15,10))
for accent in check:
      plt.plot(y_pos, MI_layers[accent],'-o', alpha=0.7, label =accent)
plt.xticks(y_pos, objects)
plt.ylim(ymin = 0)
#plt.axhline(14.28, linewidth=1, color='k')
plt.grid()
plt.ylabel('Mutual Information')
plt.xlabel('layers')
plt.title('Mututal Information across accents')
plt.legend()
plt.show()


### MI between representations and accents

In [ ]:
layers = ['spec','conv','rnn_0', 'rnn_1', 'rnn_2', 'rnn_3', 'rnn_4']
num_clust = 10000
MI_layers = []
for layer in layers:
    X_dict, y_dict, z_dict = load_data(phones,check,layer)
    X = []
    y = []
    z = []
#     for a in check:
#         X.extend(X_dict[a])
#         y.extend(y_dict[a])
    X = np.array(X)
    y = np.array(y)
    print(X.shape)
    print(y.shape)
    kmeans =  MiniBatchKMeans(n_clusters=num_clust).fit(X)
    prior_prob = np.ones(len(check))/len(check)
    print(prior_prob)
    posterior = {i:np.zeros(num_clust) for i in check}
    # compute posterior
    for a in check:
        clusters = kmeans.predict(X_dict[a])
        unique, counts = np.unique(clusters, return_counts=True)
    #     print(unique)
    #     print(counts)
        counts = counts/sum(counts)
        #print(unique.shape)
        for j in range(unique.shape[0]):
            #print(j)
    #         print(np.where(unique == j)[0])
            posterior[a][unique[j]] = counts[j]
    #     print(posterior[a])
    #     break
    joint = {i:posterior[i]*prior_prob[check.index(i)] for i in check}
    marginal_y = np.zeros(num_clust)
    for i in range(num_clust):
        for a in check:
            marginal_y[i] += joint[a][i]
    MI = 0
    eps = 10e-12
    for i in range(num_clust):
        for a in check:
            #print(joint[a][i])
    #         fr = joint[a][i]/(prior_prob[check.index[a]]*marginal(y))
    #         print(fr)
    #         print(joint[a][i])
    #         print(log(eps + joint[a][i]/(prior_prob[check.index(a)]*marginal_y[i] + eps), 2))
            MI += joint[a][i]*log((joint[a][i] + eps)/(prior_prob[check.index(a)]*marginal_y[i] + eps), 2)
    print(layer, MI)
    MI_layers.append(MI)


#print(posterior)


    

In [19]:
print(MI_layers)
# 100000 [0.3537750732377461, 0.21033976202033164, 0.2809791360718149, 0.227060025915328, 0.2452155197747754, 0.27877079278125344, 0.29850526839640545]

[0.3537750732377461, 0.21033976202033164, 0.2809791360718149, 0.227060025915328, 0.2452155197747754, 0.27877079278125344, 0.29850526839640545]
